In [ ]:
# Data is not disclosed since its given by Client.
# Some parts of code may not available but you get the idea about QAT using TF

In [1]:
!pip install tensorflow-model-optimization


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [10]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from os.path import join
from os import listdir

import cv2
import numpy as np

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D, concatenate, \
    GlobalAveragePooling2D, Dense, Input, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.utils import get_file, Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.image import resize
from tensorflow import sqrt
from tensorflow.keras.losses import mean_squared_error

import tensorflow_model_optimization as tfmot

2023-09-15 22:32:12.890616: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-15 22:32:12.921576: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_root = "data"
images_path = join(data_root, "images")
labels_path = join(data_root, "labels")

images_train_path = join(images_path, "train")
images_val_path = join(images_path, "val")

labels_train_path = join(labels_path, "train")
labels_val_path = join(labels_path, "val")

In [3]:
max_obj = 30
image_size = (299, 299)
input_shape = (299, 299, 3)
retrain_baseline = False
no_output_neurons = max_obj * 5
batch_size = 16

In [4]:
def process(img_path, lbl_path):
    labels = listdir(lbl_path)
    xy = []
    for i in range(len(labels)):
        train_label = labels[i]
        train_image = (train_label[:-4] + ".jpg")
        annotation_file_path = join(lbl_path, train_label)
        image_file_path = join(img_path, train_image)
        with open(annotation_file_path, "r") as annotation_file:
            annotations = [([float(j) for j in i.strip().split(" ")]) for i in annotation_file.readlines()]
            for annotation in annotations:
                annotation[0] = 1  # TODO since there is only one class
            while len(annotations) < max_obj:
                annotations.append([0, 0, 0, 0, 0])
            if len(annotations) > max_obj:
                annotations = annotations[:max_obj]
        annotation = np.array(annotations)
        xy.append([image_file_path, annotation.flatten()])
    return xy


In [5]:
def load_image(image_path, dsize):
    image = cv2.imread(image_path)
    image = cv2.resize(image, dsize)
    image = image.astype(float)
    image /= 255.0
    return image

In [63]:
class CustomDataGenerator(Sequence):
    def __init__(self, data, input_size, batch_size, shuffle=False):
        self.data = data
        self.input_size = input_size
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.n = len(data)
        pass

    def __get_input(self, path, target_size):
        print(path)
        image = load_img(path)
        image_arr = img_to_array(image)
        image_arr = resize(image_arr, (target_size[0], target_size[1])).numpy()
        return image_arr / 255.

    def __get_data(self, batches):
        path_batch = [i[0] for i in batches]
        y_batch = np.asarray([i[1] for i in batches])
        x_batch = np.asarray([self.__get_input(i, self.input_size) for i in path_batch])

        return x_batch, y_batch

    def on_epoch_end(self):
        if self.shuffle:
            self.data = self.data.sample(frac=1).reset_index(drop=True)

    def __getitem__(self, index):
        batches = self.data[index * self.batch_size:(index + 1) * batch_size]
        X, y = self.__get_data(batches)
        return X, y

    def __len__(self):
        return self.n // self.batch_size


In [64]:
train_dataset = process(images_train_path, labels_train_path)
val_dataset = process(images_val_path, labels_val_path)

train_generator = CustomDataGenerator(train_dataset, image_size, batch_size)
val_generator = CustomDataGenerator(val_dataset, image_size, batch_size)

In [8]:
def conv2d_bn(
        x, filters, num_row, num_col, padding="same", strides=(1, 1), name=None
):
    if name is not None:
        bn_name = name + "_bn"
        conv_name = name + "_conv"
    else:
        bn_name = None
        conv_name = None
    x = Conv2D(
        filters,
        (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name,
    )(x)
    x = BatchNormalization(axis=3, scale=False, name=bn_name)(x)
    x = Activation("relu", name=name)(x)
    return x


def InceptionV3FeatureExtractor(input_shape):
    channel_axis = 3
    img_input = Input(shape=input_shape)

    x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding="valid")
    x = conv2d_bn(x, 32, 3, 3, padding="valid")
    x = conv2d_bn(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv2d_bn(x, 80, 1, 1, padding="valid")
    x = conv2d_bn(x, 192, 3, 3, padding="valid")
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    # mixed 0: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool],
                    axis=channel_axis, name="mixed0")

    # mixed 1: 35 x 35 x 288
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool],
                    axis=channel_axis, name="mixed1")

    # mixed 2: 35 x 35 x 288
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool],
                    axis=channel_axis, name="mixed2")

    # mixed 3: 17 x 17 x 768
    branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding="valid")

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, strides=(2, 2),
                             padding="valid")

    branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = concatenate([branch3x3, branch3x3dbl, branch_pool], axis=channel_axis,
                    name="mixed3")

    # mixed 4: 17 x 17 x 768
    branch1x1 = conv2d_bn(x, 192, 1, 1)

    branch7x7 = conv2d_bn(x, 128, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

    branch7x7dbl = conv2d_bn(x, 128, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],
                    axis=channel_axis, name="mixed4")

    # mixed 5, 6: 17 x 17 x 768
    for i in range(2):
        branch1x1 = conv2d_bn(x, 192, 1, 1)

        branch7x7 = conv2d_bn(x, 160, 1, 1)
        branch7x7 = conv2d_bn(branch7x7, 160, 1, 7)
        branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

        branch7x7dbl = conv2d_bn(x, 160, 1, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 1, 7)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

        branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],
                        axis=channel_axis, name="mixed" + str(5 + i))

    # mixed 7: 17 x 17 x 768
    branch1x1 = conv2d_bn(x, 192, 1, 1)

    branch7x7 = conv2d_bn(x, 192, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 192, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

    branch7x7dbl = conv2d_bn(x, 192, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],
                    axis=channel_axis, name="mixed7")

    # mixed 8: 8 x 8 x 1280
    branch3x3 = conv2d_bn(x, 192, 1, 1)
    branch3x3 = conv2d_bn(branch3x3, 320, 3, 3, strides=(2, 2), padding="valid")

    branch7x7x3 = conv2d_bn(x, 192, 1, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 3, 3, strides=(2, 2),
                            padding="valid")

    branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = concatenate([branch3x3, branch7x7x3, branch_pool], axis=channel_axis,
                    name="mixed8")

    # mixed 9: 8 x 8 x 2048
    for i in range(2):
        branch1x1 = conv2d_bn(x, 320, 1, 1)

        branch3x3 = conv2d_bn(x, 384, 1, 1)
        branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
        branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
        branch3x3 = concatenate([branch3x3_1, branch3x3_2], axis=channel_axis,
                                name="mixed9_" + str(i), )

        branch3x3dbl = conv2d_bn(x, 448, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
        branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
        branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
        branch3x3dbl = concatenate([branch3x3dbl_1, branch3x3dbl_2],
                                   axis=channel_axis)

        branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = concatenate([branch1x1, branch3x3, branch3x3dbl, branch_pool],
                        axis=channel_axis, name="mixed" + str(9 + i))

    # # classification layer
    # x = GlobalAveragePooling2D(name="avg_pool")(x)
    # x = Dense(
    #     1000, activation="softmax", name="predictions"
    # )(x)
    # model = Model(img_input, x, name="inception_v3")

    WEIGHTS_PATH = "https://storage.googleapis.com/tensorflow/keras-applications/inception_v3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
    weights_path = get_file("inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5", WEIGHTS_PATH,
                            cache_subdir="models", file_hash="bcbd6486424b2319ff4ef7d526e38f63")

    model = Model(img_input, x, name="inception_v3")
    model.load_weights(weights_path)
    return model


In [9]:
def InceptionV3(input_shape):
    channel_axis = 3
    img_input = Input(shape=input_shape)

    x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding="valid")
    x = conv2d_bn(x, 32, 3, 3, padding="valid")
    x = conv2d_bn(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv2d_bn(x, 80, 1, 1, padding="valid")
    x = conv2d_bn(x, 192, 3, 3, padding="valid")
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    # mixed 0: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool],
                    axis=channel_axis, name="mixed0")

    # mixed 1: 35 x 35 x 288
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool],
                    axis=channel_axis, name="mixed1")

    # mixed 2: 35 x 35 x 288
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool],
                    axis=channel_axis, name="mixed2")

    # mixed 3: 17 x 17 x 768
    branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding="valid")

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, strides=(2, 2),
                             padding="valid")

    branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = concatenate([branch3x3, branch3x3dbl, branch_pool], axis=channel_axis,
                    name="mixed3")

    # mixed 4: 17 x 17 x 768
    branch1x1 = conv2d_bn(x, 192, 1, 1)

    branch7x7 = conv2d_bn(x, 128, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

    branch7x7dbl = conv2d_bn(x, 128, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],
                    axis=channel_axis, name="mixed4")

    # mixed 5, 6: 17 x 17 x 768
    for i in range(2):
        branch1x1 = conv2d_bn(x, 192, 1, 1)

        branch7x7 = conv2d_bn(x, 160, 1, 1)
        branch7x7 = conv2d_bn(branch7x7, 160, 1, 7)
        branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

        branch7x7dbl = conv2d_bn(x, 160, 1, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 1, 7)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

        branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],
                        axis=channel_axis, name="mixed" + str(5 + i))

    # mixed 7: 17 x 17 x 768
    branch1x1 = conv2d_bn(x, 192, 1, 1)

    branch7x7 = conv2d_bn(x, 192, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 192, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

    branch7x7dbl = conv2d_bn(x, 192, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],
                    axis=channel_axis, name="mixed7")

    # mixed 8: 8 x 8 x 1280
    branch3x3 = conv2d_bn(x, 192, 1, 1)
    branch3x3 = conv2d_bn(branch3x3, 320, 3, 3, strides=(2, 2), padding="valid")

    branch7x7x3 = conv2d_bn(x, 192, 1, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 3, 3, strides=(2, 2),
                            padding="valid")

    branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = concatenate([branch3x3, branch7x7x3, branch_pool], axis=channel_axis,
                    name="mixed8")

    # mixed 9: 8 x 8 x 2048
    for i in range(2):
        branch1x1 = conv2d_bn(x, 320, 1, 1)

        branch3x3 = conv2d_bn(x, 384, 1, 1)
        branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
        branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
        branch3x3 = concatenate([branch3x3_1, branch3x3_2], axis=channel_axis,
                                name="mixed9_" + str(i), )

        branch3x3dbl = conv2d_bn(x, 448, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
        branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
        branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
        branch3x3dbl = concatenate([branch3x3dbl_1, branch3x3dbl_2],
                                   axis=channel_axis)

        branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = concatenate([branch1x1, branch3x3, branch3x3dbl, branch_pool],
                        axis=channel_axis, name="mixed" + str(9 + i))

    x = Flatten()(x)
    x = Dense(256, activation="relu")(x)
    x = Dense(no_output_neurons, activation="sigmoid")(x)

    model = Model(img_input, x, name="inception_v3")
    return model


In [10]:
f_model = InceptionV3FeatureExtractor((299, 299, 3))

2023-09-15 22:32:15.054455: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-15 22:32:15.073336: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-15 22:32:15.073456: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [11]:
model = InceptionV3((299, 299, 3))

In [12]:
weights_list = f_model.get_weights()

for i, l in enumerate(f_model.layers):
    w = f_model.layers[i].weights
    model.layers[i].set_weights(w)

In [13]:
def rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))

In [14]:
quantize_model = tfmot.quantization.keras.quantize_model

In [15]:
q_model = quantize_model(model)

In [16]:
q_model.compile(optimizer="adam", loss=rmse, metrics=["mae"])

In [17]:
q_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 quantize_layer (QuantizeLa  (None, 299, 299, 3)          3         ['input_2[0][0]']             
 yer)                                                                                             
                                                                                                  
 quant_conv2d_94 (QuantizeW  (None, 149, 149, 32)         929       ['quantize_layer[0][0]']      
 rapperV2)                                                                                        
                                                                                       

In [18]:
hist = q_model.fit(train_generator, validation_data=val_generator, epochs=50)

Epoch 1/50


2023-09-15 22:32:40.827417: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2023-09-15 22:32:40.906163: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-15 22:32:43.453989: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-15 22:32:43.469585: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f11349af390 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-15 22:32:43.469606: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-09-15 22:32:43.472561: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-15 22:32:43.51565

11/11 [==============================] - 41s 855ms/step - loss: 0.3421 - mae: 0.2285 - val_loss: 0.2086 - val_mae: 0.1161
Epoch 2/50
11/11 [==============================] - 6s 549ms/step - loss: 0.2632 - mae: 0.1601 - val_loss: 0.1858 - val_mae: 0.0965
Epoch 3/50
11/11 [==============================] - 6s 556ms/step - loss: 0.2599 - mae: 0.1560 - val_loss: 0.1846 - val_mae: 0.0960
Epoch 4/50
11/11 [==============================] - 6s 570ms/step - loss: 0.2655 - mae: 0.1567 - val_loss: 0.1850 - val_mae: 0.0913
Epoch 5/50
11/11 [==============================] - 6s 557ms/step - loss: 0.2641 - mae: 0.1548 - val_loss: 0.1828 - val_mae: 0.0882
Epoch 6/50
11/11 [==============================] - 6s 572ms/step - loss: 0.2653 - mae: 0.1546 - val_loss: 0.1814 - val_mae: 0.0859
Epoch 7/50
11/11 [==============================] - 6s 555ms/step - loss: 0.2693 - mae: 0.1556 - val_loss: 0.1805 - val_mae: 0.0838
Epoch 8/50
11/11 [==============================] - 6s 565ms/step - loss: 0.2660 - mae

In [19]:
q_model.save("q_model")

INFO:tensorflow:Assets written to: q_model/assets


INFO:tensorflow:Assets written to: q_model/assets


In [27]:
from tensorflow import lite

In [28]:
converter = lite.TFLiteConverter.from_keras_model(q_model)
converter.optimizations = [lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpx11zt6s6/assets


INFO:tensorflow:Assets written to: /tmp/tmpx11zt6s6/assets
/home/gk/python-envs/scratch/lib/python3.11/site-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2023-09-15 22:50:08.925962: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-09-15 22:50:08.925985: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-09-15 22:50:08.926618: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpx11zt6s6
2023-09-15 22:50:08.993181: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-09-15 22:50:08.993211: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmpx11zt6s6
2023-09-15 22:50:09.149900: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization 

In [46]:
def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]
    for i in val_generator:
        for j in range(len(i[0])):
            image = np.expand_dims(i[0][j], axis=0)
            target = i[1][j]
            interpreter.set_tensor(input_index, image)
            interpreter.invoke()
            output = interpreter.tensor(output_index)
            print(output()[0])
    # for i, test_image in enumerate(val_dataset):
    #   if i % 1000 == 0:
    #     print('Evaluated on {n} results so far.'.format(n=i))
    #   # Pre-processing: add batch dimension and convert to float32 to match with
    #   # the model's input data format.
    #   test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    #   interpreter.set_tensor(input_index, test_image)
    # 
    #   # Run inference.
    #   interpreter.invoke()
    # 
    #   # Post-processing: remove batch dimension and find the digit with highest
    #   # probability.
    #   output = interpreter.tensor(output_index)
    #   digit = np.argmax(output()[0])
    #   prediction_digits.append(digit)
    # 
    # print('\n')
    # # Compare prediction results with ground truth labels to calculate accuracy.
    # prediction_digits = np.array(prediction_digits)
    # accuracy = (prediction_digits == test_labels).mean()
    # return accuracy


evaluate_model(interpreter)

[0.99609375 0.         0.99609375 0.         0.99609375 0.99609375
 0.         0.99609375 0.         0.99609375 0.99609375 0.
 0.99609375 0.         0.99609375 0.99609375 0.99609375 0.99609375
 0.         0.99609375 0.99609375 0.         0.99609375 0.
 0.         0.99609375 0.99609375 0.         0.         0.
 0.         0.         0.         0.         0.         0.99609375
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.       

In [29]:
interpreter = lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

# test_accuracy = evaluate_model(interpreter)
# 
# print('Quant TFLite test_accuracy:', test_accuracy)
# print('Quant TF test accuracy:', q_aware_model_accuracy)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [68]:
pred = q_model.predict(val_generator)

data/images/val/273271,1b9eb00089049cd6.jpg
data/images/val/273271,1bb400001cf13c4b.jpg
data/images/val/273271,1bab200041e8121f.jpg
data/images/val/273271,1c3a7000ee250436.jpg
data/images/val/273271,1a27b000f0c7a077.jpg
data/images/val/273271,1c02a00058d9f56e.jpg
data/images/val/273271,1bcf6000b9a2c34e.jpg
data/images/val/273271,1bdb20008371af00.jpg
data/images/val/273271,1c5ed000d09f3a10.jpg
data/images/val/273271,1b86f000bc5b77bf.jpg
data/images/val/273271,1bc75000ad1a58db.jpg
data/images/val/273271,1ba93000e00e011c.jpg
data/images/val/273271,1a02900084ed5ae8.jpg
data/images/val/273271,1c3c10003afd20b9.jpg
data/images/val/273271,1ba470008722b054.jpg
data/images/val/273271,1a51000317893a5.jpg
data/images/val/273271,1b9eb00089049cd6.jpg
data/images/val/273271,1bb400001cf13c4b.jpg
data/images/val/273271,1bab200041e8121f.jpg
data/images/val/273271,1c3a7000ee250436.jpg
data/images/val/273271,1a27b000f0c7a077.jpg
data/images/val/273271,1c02a00058d9f56e.jpg
data/images/val/273271,1bcf6000b9

In [85]:
for i in pred:
    bboxes = [bbox[1:] for bbox in i.reshape((30, 5)) if bbox[0] >= .5]
    with open("output.txt", "w") as file:
        for row in bboxes:
            row_str = " ".join([f"{item:.50f}" for item in row])  # Format each number in the row with 2 decimal places and join with spaces
            file.write(row_str + "\n")  
        file.close()
    break
# Open the file for reading
with open("output.txt", "r") as file:
    for line in file:
        print(line.strip())  # Use strip() to remove leading/trailing whitespace and newline characters

0.00004357758371043018996715545654296875000000000000 1.00000000000000000000000000000000000000000000000000 0.00000000000000000000000000000000000000000000000000 1.00000000000000000000000000000000000000000000000000
0.00000000000000000000000000000000000000000000000000 1.00000000000000000000000000000000000000000000000000 0.00000000000000000000000000000000000000000000000000 1.00000000000000000000000000000000000000000000000000
0.00000000000000000000000000000009782478093475286509 1.00000000000000000000000000000000000000000000000000 0.00000000000000000000000000000000000000000000000000 1.00000000000000000000000000000000000000000000000000
1.00000000000000000000000000000000000000000000000000 1.00000000000000000000000000000000000000000000000000 0.00000000000000000000000000000000000000000000000000 1.00000000000000000000000000000000000000000000000000
0.00000000000000000000000000000000000000000000000000 1.00000000000000000000000000000000000000000000000000 0.00000000000000000000000000000000000000000000